# Création DataFrames DOCUMENTS-SAMPLES

## README
Ce notebook permet de générer des DataFrames composées d'un échantillon des données contenues dans la DataFrame DOCUMENTS.
L'objectif de ces échantillons est de permettre de développer, tester ou appliquer des traitements sur une partie des données, afin de réduire les performances nécessaires, en termes de temps de calcul et d'espace disque requis notamment, afin de rendre possibles certaines opérations en dépit des ressources contraintes qui sont disponibles pour le projet.

Il réalise tout d'abord certaines opérations préalables (chapitre 1), dont la configuration des répertoires du projet.

A l'issue (chapitre 2), il permet de créer, pour 5 tailles prédéfinies, 5 échantillons de la DataFrame documents (25 DataFrames en tout).
Chaque DataFrame conserve les mêmes proprtions de targets et de train/val/test que la DataFrame d'origine. Les Dataframe sont nommées ainsi: 

DOCUMENTS_SAMPLE_[x]K_[n], où:
- x représente la taille de la DataFrame, en milliers de ligne prend des valeurs parmi {1, 4, 10, 40, 100};
- n est un nombre entier entre 1 et 5, permettant d'identifier l'échantillon.

## 1. Préparation

In [ ]:
import sys
from pathlib import Path

project_root = Path().resolve().parent
if not project_root in [Path(p).resolve() for p in sys.path]:
    sys.path.append(str(project_root))

from src import PATHS

In [ ]:
import os
import itertools
import pandas as pd
from sklearn.model_selection import train_test_split

## 2. Création des DataFrames DOCUMENTS_SAMPLE

### 2.1 Chargement et ajout d'une colonne stratifier
On ajouter cette colonne pour pouvoir l'utiliser ensuite lors de la génération des échantillons, afin de conserver, à la fois la proportion de train/val/test et celle de target. Naturellement, la colonne sera ensuite retirée de la DataFrame échantillon.

In [ ]:
df = pd.read_parquet(PATHS.metadata / "df_data_sets.parquet").\
    join(pd.read_parquet(PATHS.metadata / "df_encoded_labels.parquet"))
df['stratifier'] = df.data_set + '/' + df.label.astype(str)
df.head()

### 2.2 Création et sauvegarde des échantillons 

In [ ]:
sizes_k = [1, 4, 10, 40, 100]
identifiers = list(range(1, 6))

for random_state, (size, identifier) in enumerate(itertools.product(sizes_k, identifiers)):
    sample_name = f"df_documents_sample_{size}k_{identifier}.parquet"
    sample, _ = train_test_split(df, stratify=df.stratifier, train_size= 1000 * size, random_state=random_state)
    sample[[]].to_parquet(PATHS.samples / sample_name)

In [ ]:
# Verification
# df = pd.read_parquet(os.path.join(extracted_data_path, "df_documents_sample_100k_5.parquet"))
# print(df.data_set.value_counts())
# print(df.label.value_counts())

In [ ]:
pd.read_parquet(os.path.join(PATHS.samples / "df_documents_sample_1k_3.parquet")).head()